In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,44517
2,Huelva,Confirmados PDIA 14 días,1211
3,Huelva,Tasa PDIA 14 días,"235,98417678352203"
4,Huelva,Confirmados PDIA 7 días,450
5,Huelva,Tasa PDIA 7 dias,"87,69023910205195"
6,Huelva,Total Confirmados,44716
7,Huelva,Curados,40159
8,Huelva,Fallecidos,403


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  44517.0


/tmp/ipykernel_5104/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12358.0


/tmp/ipykernel_5104/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_5104/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_5104/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_5104/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1410 personas en los últimos 7 días 

Un positivo PCR cada 537 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,44517.0,450.0,1211.0,513170.0,87.690239,235.984177,154.0
Huelva-Costa,26295.0,267.0,759.0,289548.0,92.212690,262.132703,91.0
Condado-Campiña,13867.0,132.0,332.0,156231.0,84.490274,212.505841,40.0
Huelva (capital),12358.0,102.0,268.0,143837.0,70.913604,186.322017,37.0
Sierra de Huelva-Andévalo Central,3940.0,47.0,105.0,67391.0,69.742250,155.807155,19.0
Isla Cristina,3052.0,44.0,106.0,21393.0,205.674753,495.489179,18.0
Almonte,2240.0,22.0,51.0,24507.0,89.770270,208.103807,13.0
Lepe,2954.0,25.0,90.0,27880.0,89.670014,322.812052,12.0
Moguer,1919.0,23.0,55.0,21867.0,105.181323,251.520556,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cañaveral de León,30.0,4.0,9.0,396.0,1010.101010,2272.727273,1.0
Cabezas Rubias,52.0,1.0,8.0,706.0,141.643059,1133.144476,0.0
Escacena del Campo,231.0,4.0,21.0,2287.0,174.901618,918.233494,0.0
Rosal de la Frontera,71.0,4.0,14.0,1697.0,235.710077,824.985268,0.0
Sanlúcar de Guadiana,34.0,3.0,3.0,403.0,744.416873,744.416873,0.0
San Silvestre de Guzmán,21.0,4.0,4.0,614.0,651.465798,651.465798,0.0
San Bartolomé de la Torre,374.0,15.0,21.0,3761.0,398.830098,558.362138,3.0
Lucena del Puerto,362.0,7.0,17.0,3261.0,214.658080,521.312481,1.0
Isla Cristina,3052.0,44.0,106.0,21393.0,205.674753,495.489179,18.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Bonares,451.0,1.0,9.0,6060.0,16.501650,148.514851,0.0,0.111111
Cabezas Rubias,52.0,1.0,8.0,706.0,141.643059,1133.144476,0.0,0.125000
Valverde del Camino,829.0,2.0,14.0,12750.0,15.686275,109.803922,0.0,0.142857
Escacena del Campo,231.0,4.0,21.0,2287.0,174.901618,918.233494,0.0,0.190476
San Juan del Puerto,751.0,5.0,25.0,9411.0,53.129317,265.646584,1.0,0.200000
Villanueva de los Castillejos,206.0,2.0,9.0,2825.0,70.796460,318.584071,1.0,0.222222
Zalamea la Real,113.0,1.0,4.0,3054.0,32.743942,130.975769,0.0,0.250000
Punta Umbría,1246.0,11.0,41.0,15355.0,71.637903,267.014002,1.0,0.268293
Lepe,2954.0,25.0,90.0,27880.0,89.670014,322.812052,12.0,0.277778
